# Домашняя работа 7. Пропущенные значения

Вариант с кредитами.

Лазарев Эдуард Артемович
N33471

In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [12, 12]

!pip install impyute
import impyute as impy
from scipy import stats

!pip install scikit-learn==0.22.1
!pip install impyute
import impyute as impy
!pip install missingpy
from missingpy import MissForest

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Загружаем данные
df = pd.read_csv("default of credit card clients.csv", decimal=",", delimiter=";")
len(df)

30000

Проверяем правильность данных

In [3]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [4]:
df.dtypes

ID                            int64
LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_0                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
default_payment_next_month    int64
dtype: object

Выделяем часть датасета для работы

In [5]:
X = df.iloc[:, :-1].values  

Заменяем значения 

In [6]:
def make_missing_value(X, del_fraction):
    N = X.shape[0]
    D = X.shape[1]
    delete_mask = np.array(np.random.random((N, D)))
    #Заполняем обозначенную долю True - будет nan, остальное False 
    #while (sum(sum(delete_mask))!= del_fraction*N*D): первый вариант не вариант, слишком долго
     # delete_mask = np.array(np.random.random((N, D)) < del_fraction)
    mas = []
    for i in range(30000):
      if i < int(30000*0.1):
        mas.append(bool(True))
      else:
        mas.append(bool(False))
    delete_mask = pd.DataFrame()
    for i in range (24):
      np.random.shuffle(mas)
      delete_mask[i] = mas

    # Значения где был True заменяются на nan, остальное остается значениями
    new_X = np.array(X)
    new_X = new_X.astype('float64')
    new_X[delete_mask] = np.nan
    return new_X

"Удаляем" долю результатов и при помощи impyute заменяем их на предполагаемые значения. Считаем точность, СКО, дисперсию. Применяем данную схему для заполнения средним и knn для всех вариантов количества пропусков.

In [7]:
tt = X
tt = make_missing_value(tt, 0.1)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_10_means = (30000*24*0.1-(30000*24 - np.sum(y)))/(30000*24*0.1)
disp_10_means = np.var(means)
sko_10_means = np.std(means)

In [8]:
tt = X
tt = make_missing_value(tt, 0.1)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_10_knn = (30000*24*0.1-(30000*24 - np.sum(y)))/(30000*24*0.1)
disp_10_knn = np.var(knn)
sko_10_knn = np.std(knn)

In [9]:
tt = X
tt = make_missing_value(tt, 0.1)
imputer = MissForest()
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_10_rf = (30000*24*0.1-(30000*24 - np.sum(y)))/(30000*24*0.1)
disp_10_rf = np.var(rf)
sko_10_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


Повторяем для 25%

In [10]:
tt = X
tt = make_missing_value(tt, 0.25)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_25_means = (30000*24*0.25-(30000*24 - np.sum(y)))/(30000*24*0.25)
disp_25_means = np.var(means)
sko_25_means = np.std(means)

In [11]:
tt = X
tt = make_missing_value(tt, 0.25)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_25_knn = (30000*24*0.25-(30000*24 - np.sum(y)))/(30000*24*0.25)
disp_25_knn = np.var(knn)
sko_25_knn = np.std(knn)

In [12]:
tt = X
tt = make_missing_value(tt, 0.25)
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_25_rf = (30000*24*0.25-(30000*24 - np.sum(y)))/(30000*24*0.25)
disp_25_rf = np.var(rf)
sko_25_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7


Повторяем для 30%

In [13]:
tt = X
tt = make_missing_value(tt, 0.3)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_30_means = (30000*24*0.3-(30000*24 - np.sum(y)))/(30000*24*0.3)
disp_30_means = np.var(means)
sko_30_means = np.std(means)

In [14]:
tt = X
tt = make_missing_value(tt, 0.3)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_30_knn = (30000*24*0.3-(30000*24 - np.sum(y)))/(30000*24*0.3)
disp_30_knn = np.var(knn)
sko_30_knn = np.std(knn)

In [15]:
tt = X
tt = make_missing_value(tt, 0.3)
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_30_rf = (30000*24*0.3-(30000*24 - np.sum(y)))/(30000*24*0.3)
disp_30_rf = np.var(rf)
sko_30_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


Сводня таблица результатов 

In [16]:
table = pd.DataFrame(index = ['means, 10%', 'knn, 10%','rf, 10%', 'means, 25%', 'knn, 25%','rf, 25%',
                              'means, 30%', 'knn, 30%', 'rf, 30%'])

table['acc'] = [acc_10_means, acc_10_knn, acc_10_rf,
                acc_25_means, acc_25_knn, acc_25_rf,
                acc_30_means, acc_30_knn, acc_30_rf]
table['disp'] = [disp_10_means, disp_10_knn, disp_10_rf,
                 disp_25_means, disp_25_knn, disp_25_rf,
                 disp_30_means, disp_30_knn, disp_30_rf]
table['sko'] = [sko_10_means, sko_10_knn, sko_10_rf,
                sko_25_means, sko_25_knn,sko_25_rf,
                sko_30_means, sko_30_knn, sko_30_rf]
print('sko X ', np.std(X),'  disp X', np.var(X))
table

sko X  56435.51531636148   disp X 3184967389.0232716


,acc,disp,sko
"means, 10%",0.000000,2.998826e+09,54761.536090
"knn, 10%",0.074722,3.001141e+09,54782.674360
"rf, 10%",0.180653,3.138654e+09,56023.689970
"means, 25%",0.600000,2.992502e+09,54703.765770
"knn, 25%",0.630261,2.996561e+09,54740.856808
"rf, 25%",0.672517,3.143376e+09,56065.818604
"means, 30%",0.666667,3.000109e+09,54773.246300
"knn, 30%",0.691815,3.013617e+09,54896.424429
"rf, 30%",0.726528,3.144240e+09,56073.526966


## Вывод

По результатам лабораторной можно сказать, что случайный лес работает лучше всего. Также многое зависит от количества пропущенных значений. Для качественного заполнения их должно быть не мало (больше 25%), но и не много (при более 50% пропусков не имеет смысла их заполнять), иначе заполненные значения будут либо сильно отличаться от искомых.